In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re
from zipfile import ZipFile
import gzip

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [9]:
def decompress_data(file, csv, index_col=False, type='zip'):
    if type == 'zip':
        with ZipFile(f'data/{file}.zip') as archive:
            file = archive.open(f'{csv}.csv')
            df = pd.read_csv(file, index_col=index_col)
            file.close()
    elif type == 'gz':
        with gzip.open(f'data/{file}.csv.gz') as archive:
            df = pd.read_csv(archive, index_col=index_col)
    else:
        df = None
    return df

prefix = 'drive/MyDrive/data/'

zips = ['google_play', 'google_play_2016', 'steam']
csvs = [
    [
        'Google-Playstore'
    ],
    [
        'latest-with-added-date'
    ],
    [
        'steam',
        'steam_description_data',
        'steam_media_data',
        'steam_requirements_data',
        'steam_support_info',
        'steamspy_tag_data'
    ]
]

In [ ]:
gp21_df = decompress_data(zips[0],csvs[0][0])

In [10]:
gp16_df = decompress_data(zips[1],csvs[1][0],type='gz')

KeyboardInterrupt: 

In [16]:
gp21_df.head()

,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,...,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,Scraped Time
0,Gakondo,com.ishakwe.gakondo,Adventure,0.0,0.0,10+,10.0,15,True,0.0,...,https://beniyizibyose.tk/#/,jean21101999@gmail.com,"Feb 26, 2020","Feb 26, 2020",Everyone,https://beniyizibyose.tk/projects/,False,False,False,2021-06-15 20:19:35
1,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4.4,64.0,"5,000+",5000.0,7662,True,0.0,...,https://webserveis.netlify.app/,webserveis@gmail.com,"May 21, 2020","May 06, 2021",Everyone,https://dev4phones.wordpress.com/licencia-de-uso/,True,False,False,2021-06-15 20:19:35
2,Vibook,com.doantiepvien.crm,Productivity,0.0,0.0,50+,50.0,58,True,0.0,...,NaN,vnacrewit@gmail.com,"Aug 9, 2019","Aug 19, 2019",Everyone,https://www.vietnamairlines.com/vn/en/terms-an...,False,False,False,2021-06-15 20:19:35
3,Smart City Trichy Public Service Vehicles 17UC...,cst.stJoseph.ug17ucs548,Communication,5.0,5.0,10+,10.0,19,True,0.0,...,http://www.climatesmarttech.com/,climatesmarttech2@gmail.com,"Sep 10, 2018","Oct 13, 2018",Everyone,NaN,True,False,False,2021-06-15 20:19:35
4,GROW.me,com.horodyski.grower,Tools,0.0,0.0,100+,100.0,478,True,0.0,...,http://www.horodyski.com.pl,rmilekhorodyski@gmail.com,"Feb 21, 2020","Nov 12, 2018",Everyone,http://www.horodyski.com.pl,False,False,False,2021-06-15 20:19:35


In [ ]:
gp21_df['Category'].value_counts()

In [31]:
gaming_categories = [
    'Arcade',
    'Puzzle',
    'Casual',
    'Action',
    'Simulation',
    'Adventure',
    'Board',
    'Racing',
    'Role Playing',
    'Strategy',
    'Card'
]

In [33]:
# Include only games
gp21_df = gp21_df[gp21_df['Category'].isin(gaming_categories)]

In [17]:
gp18_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [ ]:
gp18_df['Category'].value_counts()

In [34]:
gp18_df['Installs'].value_counts()

1,000,000+        1579
10,000,000+       1252
100,000+          1169
10,000+           1054
1,000+             907
5,000,000+         752
100+               719
500,000+           539
50,000+            479
5,000+             477
100,000,000+       409
10+                386
500+               330
50,000,000+        289
50+                205
5+                  82
500,000,000+        72
1+                  67
1,000,000,000+      58
0+                  14
0                    1
Free                 1
Name: Installs, dtype: int64

In [46]:
steam_df = decompress_data(zips[2],csvs[2][0])
steam_df.set_index('appid', inplace=True)

In [48]:
steam_lost_df = pd.read_csv(f'{prefix}steam_lost.csv', encoding='unicode_escape')
steam_lost_df.set_index('AppID', inplace=True)

In [49]:
steam_df.head()

,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
appid,,,,,,,,,,,,,,,,,
10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [51]:
steam_lost_df.head()

,Owners,Name,Type,Changed,Kinguin,Achievements
AppID,,,,,,
43110,77.48%,Metro 2033,Purchase disabled,03/2021,15.89,48
35420,71.18%,Killing Floor Mod: Defence Alliance 2,Purchase disabled,09/2022,NaN,30
310560,68.04%,DiRT Rally,Purchase disabled,11/2022,NaN,170
43160,67.58%,Metro: Last Light Complete Edition,Delisted,05/2023,9.07,70
863550,67.35%,HITMAN? 2,Purchase disabled,01/2023,NaN,146


In [56]:
steam_df.join(steam_lost_df, how='right')

,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,...,average_playtime,median_playtime,owners,price,Owners,Name,Type,Changed,Kinguin,Achievements
AppID,,,,,,,,,,,,,,,,,,,,,
43110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,77.48%,Metro 2033,Purchase disabled,03/2021,15.89,48
35420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,71.18%,Killing Floor Mod: Defence Alliance 2,Purchase disabled,09/2022,NaN,30
310560,DiRT Rally,2015-12-07,1.0,Codemasters Racing Studio;Feral Interactive (L...,Codemasters;Feral Interactive (Linux);Feral In...,windows;mac;linux,0.0,Single-player;Multi-player;Steam Achievements;...,Racing;Simulation;Sports,Racing;Simulation;Driving,...,488.0,251.0,1000000-2000000,24.99,68.04%,DiRT Rally,Purchase disabled,11/2022,NaN,170
43160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,67.58%,Metro: Last Light Complete Edition,Delisted,05/2023,9.07,70
863550,HITMAN™ 2,2018-11-13,1.0,IO Interactive A/S,Warner Bros Interactive Entertainment,windows,0.0,Single-player;Online Multi-Player;Online Co-op...,Action,Stealth;Action;Assassin,...,636.0,514.0,5000000-10000000,44.99,67.35%,HITMAN? 2,Purchase disabled,01/2023,NaN,146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2297550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.01%,F1 23 Beta,Test app,01/2023,NaN,?
2299850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.01%,ERA-????,Unreleased,02/2023,NaN,?
2351270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.01%,Team Disorder Hodon UD,Purchase disabled,06/2023,NaN,?


In [61]:
no_of_steam_games = steam_df.shape[0]
no_of_lost_steam_games = steam_lost_df.shape[0]

funnel_data = dict(
    number=[no_of_steam_games, no_of_lost_steam_games],
    stage=['Current Steam Games', 'Lost Steam Games']
)
fig = px.funnel(funnel_data, x='number', y='stage')
fig.show()